## 감정 분석
항공사 리뷰 데이터로 감정 분석하기 (트위터에 데이터 사용)

In [3]:
import pandas as pd
import re, os
pd.set_option('display.max_colwidth', 240)

In [56]:
PATH = '../data/'
FILE = os.path.join(PATH, 'es-airlines-tweets', 'tweets_public.csv')

tweets = pd.read_csv(FILE)
tweets.sample(1)

,airline_sentiment,is_reply,reply_count,retweet_count,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
5741,positive,False,0,0,Aerolínea Ryanair reconoce sindicatos para evitar huelgas https://t.co/eyDhhQW778,NaN,Fri Dec 15 13:51:03 +0000 2017,941666919534313473,NaN,Pacific Time (US & Canada)


In [57]:
# text 정제를 위해서 text 열만 골라냄ㄴ
tweets = tweets.loc[:, ['text', 'airline_sentiment']]
tweets.sample(5)

,text,airline_sentiment
5733,@Iberia FAVOR COMPARTIR. QUE PASA CON IBERIA QUE COBRAN INSTANTANEAMENTE LAS VENTAS PERO CUANDO TIENEN QUE DEVOLVER… https://t.co/wXZsRibIju,negative
1796,"@Iberia Hace cuatro días me dicen que procedían a darme la indemnizaciòn y aquí sigo esperándola... Desde el mes de agosto, vergonzoso.",negative
6006,@ivan_cortina @Iberia A @SupermanlopezN le dañaron un trofeo y la bici llego días después generando descanso obliga… https://t.co/MTiqubUFSa,negative
6106,"Otra cosa, Ryanair, ya me puedes sentar sola en un avión lleno de bebés que no pienso pagar por elegir asiento.",negative
5406,"Unas idiotas se me han colado, pero les han hecho dejar el equipaje de mano en la bodega. Llámalo karma, ¡yo lo llamo Ryanair! 🙋🏻‍♂️",negative


In [58]:
# @장소를 다른 열로 저장하기
tweets['at'] = tweets.text.str.extract(r'^(@\S+)')
tweets.sample(5)

,text,airline_sentiment,at
455,@Iberia por qué no puedo hacer el checking de mi vuelo IB3167? https://t.co/cauYpRzU2I,neutral,@Iberia
2253,Iberia aumentará sus vuelos semanales de la ruta Buenos Aires-Madrid en 2018 https://t.co/pfQQ95bOAu #NoTextAndDrive https://t.co/yu3ukqZmNK,neutral,NaN
5700,"No volveré a viajar en IBERIA, pésima atención: perdieron mi maleta y regresó después de 5 meses, con 17 kilos menos https://t.co/0i4rDE9cG9",negative,NaN
1575,@Iberia sabemos si se cancela este vuelo?? Gracias! https://t.co/YTxlmAl8Dj,neutral,@Iberia
3280,El canal Smithsonian recrea en un vídeo el accidente de Spanair en Barajas #JK5022 https://t.co/d65ds4Muic,neutral,NaN


In [59]:
# 텍스트에서 hashtag, @, http 제거하기
def remove_handles(text):
    return re.sub(r'@\S+|https://\S+|\#', '', text) # @, hashtag, address 삭제

tweets.text = tweets.text.apply(remove_handles)
tweets.sample(10)

,text,airline_sentiment,at
1190,Tenerife Iberia Toscal y Bisontes Castellón cierran este domingo la Jornada 11 en Segunda División,neutral,NaN
6357,TipicoDeNavidad perdió mi maleta hace 5 días y no saben dónde está. Tengo medicamentos que no estoy tomand…,negative,NaN
3980,Esos jeans combinaban con Alpargatas Iberia,positive,@vladyestay
1023,Bueno... puntualizar que aún no tiene tantos. Un poco cascado sí está pero no…,neutral,@slalanda
3476,estoy muy descontenta con su servicio y gestión de mi reserva de vuelo. Han contestado tarde a mi petición d…,negative,@Rumbo
1033,Qatar Airways (1º) e Iberia (2º) fueron nombradas como las aerolíneas más puntuales en el último semestre con 91.15…,positive,NaN
7029,Me ha gustado un vídeo de ( - Ryanair passenger jumps out emergency exit in Malaga (Boeing 737)).,negative,NaN
7546,"o. debería mejorar su sistema de mailings con las reservas. Aprendan un poco de al respecto, ple…",negative,NaN
3150,Para…,positive,@Grupo_Envera_
2436,"Y confirmais cómo os reis del cliente: HACEIS LLAMAR A UN NÚM DE PAGO PARA DECIR NO TE DEVUELVO NADA, felic…",negative,@Iberia


### 감정분석
- textblob: 영어밖에 지원이 안 됨. 스페인어->영어 변환 후 사용
- sentiment_analysis_spanish

In [60]:
from textblob import TextBlob

In [61]:
# es -> en 변환
def get_tweet_sentiment(tweet):
    analysis = TextBlob(tweet)
    
    language = analysis.detect_language()
    if language == 'en':
        analysis_ready = analysis
    else: # 영어가 아닌 언어는 영어로 바꿈
        analysis_ready = analysis.translate(to='en')

    if analysis_ready.sentiment.polarity > 0: # polarity를 측정하는 메소드를 가짐
        return 'positive'
    elif analysis_ready.sentiment.polarity == 0: 
        return 'neutral'
    else: 
        return 'negative'

In [62]:
# 예시
sentence = 'I hate New York.'
blob = TextBlob(sentence)
blob.sentiment # [-1, 1]

Sentiment(polarity=-0.33181818181818185, subjectivity=0.6772727272727272)

In [63]:
# 랜덤으로 한 문장 뽑기
sentence = tweets.text.sample(1).tolist()
sentence

['Iberia recibe un nuevo avión A330-200 y lo bautiza como “Santo Domingo”  ']

In [64]:
get_tweet_sentiment(sentence[0])

'positive'

#### sentiment_analysis_spanish
0~1로 표현. neg-pos

In [65]:
from sentiment_analysis_spanish import sentiment_analysis

In [66]:
#test
senti_analizer = sentiment_analysis.SentimentAnalysisSpanish()
print(senti_analizer.sentiment("Me gusta la tombola. Es genial.")) # positive

0.9304396176531412


In [67]:
tweets['polarity'] = tweets.text.apply(senti_analizer.sentiment)
tweets.head(3)

,text,airline_sentiment,at,polarity
0,Trabajar en Ryanair como TMA: empleo,neutral,NaN,0.230391
1,Cuando gusten en Cancún se viaja y disfruta de manera sin igual,neutral,@Iberia,0.025044
2,"Sabiais que te trata muy bien en santiago de chile?Te cambia el asiento,te manda a volar en el wc trasero,e…",negative,NaN,0.350941


In [68]:
# sorting
tweets.sort_values(by=['polarity'], ascending=False).head(10)

,text,airline_sentiment,at,polarity
3216,Vistas que no tenemos todos los días. Un A330 de Iberia en aproximación por el centro de Madrid,neutral,NaN,1.000000
1536,Iberia tendrá nuevas conexiones en 2018 desde el aeropuerto de Madrid hacia Managua y San Francisco…,positive,NaN,1.000000
5240,La aerolínea tendrá nuevas conexiones en 2018 desde el aeropuerto de Madrid hacia Managua y SanFrancisco,positive,NaN,1.000000
92,La aerolínea tendrá nuevas conexiones en 2018 desde el aeropuerto de Madrid hacia Managua y SanFrancisco,neutral,NaN,1.000000
6808,Samsung e Iberia sorprenden a los viajeros de un vuelo de Madrid a La Coruña con un «regalazo» vía,positive,NaN,1.000000
3403,Samsung e Iberia sorprenden a los viajeros de un vuelo de Madrid a La Coruña con un «regalazo» vía,positive,NaN,1.000000
2909,muchas gracias al personal de Business en aeropuerto de Madrid.La ayuda en situación complicada muestra su gran profesionalidad,positive,@Iberia,1.000000
5208,Iberia remodela su zona VIP del aeropuerto de Madrid con muchas comodidades:,positive,NaN,0.999999
7001,"Un muy buen destino, me encantó 👋👋👋",positive,@AParayas,0.999998
4468,Iberia inaugura un nuevo espacio Premium en el aeropuerto de Madrid,neutral,NaN,0.999998


In [52]:
#option 1. polarity 점수 구간에 따라 레이블을 positive, negative로 나누기 (2가지)

tweets['label'] = tweets.polarity.apply(lambda x: 'positive' if x >= 0.5 else 'negative')
tweets.head(5)

,text,at,polarity,label
0,Trabajar en Ryanair como TMA: empleo,NaN,0.230391,negative
1,Cuando gusten en Cancún se viaja y disfruta de manera sin igual,@Iberia,0.025044,negative
2,"Sabiais que te trata muy bien en santiago de chile?Te cambia el asiento,te manda a volar en el wc trasero,e…",NaN,0.350941,negative
3,"NUNCA NUNCA NUNCA pidáis el café de Ryanair.\nBueno, nada que vendan a bordo.",NaN,0.000305,negative
4,Muchos éxito…,@cris_tortu,0.555005,positive


In [73]:
#option 2. airline_sentiment에 따라 나누기 (3가지)

def tag_label(sentiment):
    sentiment = sentiment.lower()
    
    if sentiment == 'neutral':
        return 0
    elif sentiment == 'positive':
        return 1
    else:
        return 2
        
tweets['label'] = tweets.airline_sentiment.apply(tag_label)
tweets.head(5)

,text,airline_sentiment,at,polarity,label
0,Trabajar en Ryanair como TMA: empleo,neutral,NaN,0.230391,0
1,Cuando gusten en Cancún se viaja y disfruta de manera sin igual,neutral,@Iberia,0.025044,0
2,"Sabiais que te trata muy bien en santiago de chile?Te cambia el asiento,te manda a volar en el wc trasero,e…",negative,NaN,0.350941,2
3,"NUNCA NUNCA NUNCA pidáis el café de Ryanair.\nBueno, nada que vendan a bordo.",negative,NaN,0.000305,2
4,Muchos éxito…,positive,@cris_tortu,0.555005,1


In [74]:
tweets.label.value_counts() # 주로 부정적인 데이터가 많음.

2    3769
0    2609
1    1489
Name: airline_sentiment, dtype: int64

In [75]:
# 파일로 저장
out_file = os.path.join(PATH, 'OUT', 'tweets_public_polarity.csv')
tweets.to_csv(out_file, sep='\t', encoding='utf-8')